In [1]:
# NB: This is for spark running on parquet files converted from
#     baconbits skims. This is a prototype, there is lots of boilerplate.
#     We're making it better :-)

import pyspark.sql
import os
import sys
from pyarrow.compat import guid
from coffea.processor.spark.detail import _spark_initialize

# The following line is necessary because we're working in a
# virtualenv. Without it, executors will use the wrong interpreter!
#os.environ['PYSPARK_PYTHON'] = sys.executable

#    .config("spark.driver.extraClassPath","/home/cms.lgray/sparkMeasure/target/scala-2.12/spark-measure_2.12-0.14-SNAPSHOT.jar") \
#    .master('local[1]') \
#    .master('spark://cmsspark-submit.fnal.gov:7077') \
conf = pyspark.sql.SparkSession.builder \
    .appName('decaf-%s' % guid()) \
    .master('spark://cmsspark-submit.fnal.gov:7077') \
    .config('spark.driver.cores',8) \
    .config('spark.driver.memory','10g') \
    .config('spark.executor.memory', "20g") \
    .config('spark.debug.maxToStringFields',1000) \
    .config('spark.executorEnv.XRD_RUNFORKHANDLER','1') \
    .config('spark.executorEnv.X509_PROXY',"%s" % os.environ['X509_USER_PROXY']) \
    .config('spark.files','%s' % os.environ['X509_USER_PROXY']) \
    .config('spark.sql.execution.arrow.enabled','true') \
    .config("spark.blockManager.port", "8000") \
    .config('spark.shuffle.service.enabled','true') \
    .config('spark.scheduler.mode', 'FAIR') \
    .config('spark.executor.instances','1') \
    .config('spark.dynamicAllocation.enabled','true') \
    .config('spark.dynamicAllocation.executorIdleTimeout','1m') \
    .config('spark.dynamicAllocation.cachedExecutorIdleTimeout','10m') \
    .config('spark.dynamicAllocation.minExecutors','1') \
    .config('spark.dynamicAllocation.maxExecutors','80') \
    .config('spark.sql.execution.arrow.pyspark.enabled','true') \
    .config('spark.sql.execution.arrow.maxRecordsPerBatch', 200000) \
    .config('spark.hadoop.fs.xrootd.read.buffer', 100 * 1048576) \
    .config('spark.hadoop.fs.xrootd.write.buffer', 100 * 1048576) \
    .config('spark.driver.extraClassPath','file:///opt/hadoop-xrootd/hadoop-xrootd-1.0.4-jar-with-dependencies.jar') \
    .config('spark.executor.extraClassPath','file:///opt/hadoop-xrootd/hadoop-xrootd-1.0.4-jar-with-dependencies.jar')
session = _spark_initialize(config=conf,log_level='WARN',laurelin_version='0.4.2-SNAPSHOT')
sc = session.sparkContext
sc.setLogLevel("WARN")
spark = session

#from sparkmeasure import TaskMetrics
#taskmetrics = TaskMetrics(spark)

partitionsize = 200000
thread_workers = 32


In [2]:
import pyspark.sql.functions as fn
from tqdm import tqdm
import json

fileslice = slice(None)
with open("metadata/2018.json") as fin:
    samplefiles = json.load(fin)

filelist = {}
i = 0
for dataset, info in samplefiles.items():
    i = i + 1
    #print(i)
    if i > 10: continue
    files = []
    for file in info['files'][fileslice]:
        files.append(file)
        if 'MET____0_' in dataset: 
            filelist[dataset] = {'files': files, 'treename': 'Events'}
print(filelist['MET____0_'])


{'files': ['root://cmsxrootd.fnal.gov//store/group/lpccoffea/coffeabeans/102X/nano_2018/MET/NanoTuples-2018_Run2018A-17Sep2018-v1/190326_052413/0000/nano_1.root', 'root://cmsxrootd.fnal.gov//store/group/lpccoffea/coffeabeans/102X/nano_2018/MET/NanoTuples-2018_Run2018A-17Sep2018-v1/190326_052413/0000/nano_10.root', 'root://cmsxrootd.fnal.gov//store/group/lpccoffea/coffeabeans/102X/nano_2018/MET/NanoTuples-2018_Run2018A-17Sep2018-v1/190326_052413/0000/nano_100.root', 'root://cmsxrootd.fnal.gov//store/group/lpccoffea/coffeabeans/102X/nano_2018/MET/NanoTuples-2018_Run2018A-17Sep2018-v1/190326_052413/0000/nano_101.root', 'root://cmsxrootd.fnal.gov//store/group/lpccoffea/coffeabeans/102X/nano_2018/MET/NanoTuples-2018_Run2018A-17Sep2018-v1/190326_052413/0000/nano_102.root', 'root://cmsxrootd.fnal.gov//store/group/lpccoffea/coffeabeans/102X/nano_2018/MET/NanoTuples-2018_Run2018A-17Sep2018-v1/190326_052413/0000/nano_103.root', 'root://cmsxrootd.fnal.gov//store/group/lpccoffea/coffeabeans/102X/n

In [3]:
#get the dark Higgs analysis worker
from coffea.util import load

processor_instance = load('processors/darkhiggs2018.coffea')


In [4]:
import time
from coffea.processor import run_spark_job
from coffea.processor.spark.spark_executor import spark_executor

tic = time.time()
final_accumulator = run_spark_job(filelist, processor_instance, spark_executor, 
                                  spark=spark, partitionsize=partitionsize, thread_workers=thread_workers,
                                  executor_args={'file_type': 'root', 'cache': False})
dt = time.time() - tic


/opt/spark/python/pyspark/sql/dataframe.py:2139: UserWarning: toPandas attempted Arrow optimization because 'spark.sql.execution.arrow.enabled' is set to true, but has reached the error below and can not continue. Note that 'spark.sql.execution.arrow.fallback.enabled' does not have an effect on failures in the middle of computation.
  An error occurred while calling o796.getResult.
: org.apache.spark.SparkException: Exception thrown in awaitResult: 
	at org.apache.spark.util.ThreadUtils$.awaitResult(ThreadUtils.scala:226)
	at org.apache.spark.api.python.PythonServer.getResult(PythonRDD.scala:874)
	at org.apache.spark.api.python.PythonServer.getResult(PythonRDD.scala:870)
	at sun.reflect.NativeMethodAccessorImpl.invoke0(Native Method)
	at sun.reflect.NativeMethodAccessorImpl.invoke(NativeMethodAccessorImpl.java:62)
	at sun.reflect.DelegatingMethodAccessorImpl.invoke(DelegatingMethodAccessorImpl.java:43)
	at java.lang.reflect.Method.invoke(Method.java:498)
	at py4j.reflection.MethodInvok

Py4JJavaError: An error occurred while calling o796.getResult.
: org.apache.spark.SparkException: Exception thrown in awaitResult: 
	at org.apache.spark.util.ThreadUtils$.awaitResult(ThreadUtils.scala:226)
	at org.apache.spark.api.python.PythonServer.getResult(PythonRDD.scala:874)
	at org.apache.spark.api.python.PythonServer.getResult(PythonRDD.scala:870)
	at sun.reflect.NativeMethodAccessorImpl.invoke0(Native Method)
	at sun.reflect.NativeMethodAccessorImpl.invoke(NativeMethodAccessorImpl.java:62)
	at sun.reflect.DelegatingMethodAccessorImpl.invoke(DelegatingMethodAccessorImpl.java:43)
	at java.lang.reflect.Method.invoke(Method.java:498)
	at py4j.reflection.MethodInvoker.invoke(MethodInvoker.java:244)
	at py4j.reflection.ReflectionEngine.invoke(ReflectionEngine.java:357)
	at py4j.Gateway.invoke(Gateway.java:282)
	at py4j.commands.AbstractCommand.invokeMethod(AbstractCommand.java:132)
	at py4j.commands.CallCommand.execute(CallCommand.java:79)
	at py4j.GatewayConnection.run(GatewayConne

In [ ]:
nevt = sum(spark_executor.counts.values())
print('processed:',nevt,'events')
print('total time: ',dt/60)
print('μs/evt', dt/nevt*1e6)
print('Mevt/s', nevt/dt/1e6)


In [ ]:
from coffea import hist
from coffea.util import save
import numpy as np

nbins = sum(sum(arr.size for arr in h._sumw.values()) for h in final_accumulator.values() if isinstance(h, hist.Hist))
nfilled = sum(sum(np.sum(arr>0) for arr in h._sumw.values()) for h in final_accumulator.values() if isinstance(h, hist.Hist))
print("Processed %.1fM events" % (nevt/1e6, ))
print("Filled %.1fM bins" % (nbins/1e6, ))
print("Nonzero bins: %.1f%%" % (100*nfilled/nbins, ))

# Pickle is not very fast or memory efficient, will be replaced by something better soon
save(final_accumulator,'pods/pods2018.coffea')

#dt = time.time() - tstart
#print("%.2f us*cpu/event overall" % (1e6*dt*nworkers/final_accumulators['nentries'], ))


In [ ]:
spark.stop()
